# Project 2: Wrangle OpenStreetMap data
***

## 1. Map data
For this project I have chosen to download the data corresponding to the city of A Coruña, Spain. The motivation behind this is that this is my hometown and, therefore, I am interested to see the degree of representativeness of the OpenStreetMap data and help its improvement if needed. The file **02_Map_Description.txt** contains more information on the map used.

## 2. Data auditing
The first issue that I faced is that the data is composed mainly of Unicode strings, as the names of streets, cities and so on contain characters in Spanish which are not part of the ASCII, such as the letter ñ. For this reason, I have decided to write the code in Python 3.6, which has a simpler implementation of Unicode strings than Python 2.

### Types of tags
In terms of auditing the data, I first wanted to investigate the different types of tags that are contained in the dataset. For this, I created a Python script named **audit_tag_types.py** which provides a dictionary with all the different tag types associated to a given element type (e.g. way) and the different values they can take, as well as a dictionary with the number of occurrences for each tag type.

In [1]:
OSM_FILE = "LCG_map.osm" # Name of the .osm file containing the map

In [2]:
import audit_tag_types

node_tag_types, node_tag_numbers = audit_tag_types.get_tags(OSM_FILE, 'node')
way_tag_types, way_tag_numbers = audit_tag_types.get_tags(OSM_FILE, 'way')

In [3]:
# Print the most common tag types for ways (sorted by number of occurrences)
for key in sorted(way_tag_numbers, key=way_tag_numbers.get, reverse=True):
    if (way_tag_numbers[key] >= 200):
        print(key, '=', way_tag_numbers[key])

name = 3114
name:es = 946
name:gl = 933
addr:housenumber = 478
addr:street = 474
alt_name = 222
short_name = 220


In [4]:
for i, value in enumerate(way_tag_types["addr:street"]):
    if i >= 5:
        break
    else:
        print(value)

As Xubias de Abaixo
Praza Emilio Jaspe Deza
Praza de Ourense
Avenida Nova York
Rúa da Ribeira


The tag **"addr:street"** contains the street names for nodes and ways. A quick check showed that the street names seem to be correctly defined, with the street type given at the beginning of the street name (as it is custom in Spain).

In [5]:
for i, value in enumerate(way_tag_types["name"]):
    if i >= 5:
        break
    else:
        print(value)

Praza José González Dopeso
Parque de Novo Mesoiro
Rúa Caramuxo
Rúa Vigovidin
Parque de Marte


The tag **"name"** &ndash; the most frequent one for both nodes and ways &ndash; seem to contain not only the names of buildings and places, but also the names of streets. This generates a discrepancy, since there are two variables providing the street names. I corrected this when converting the data to CSV format.

Additionally, there are several tag types related to "name": name:es, name:gl, name:en, alt_name or short_name. The tags containing a semicolon, such as name:es, contain the name of the given element in different languages (**es** for Spanish or **en** for English), whereas the "alt_name" and "short_name" tags probably contain variations of the original name. I took this into account when converting the data to CSV format, in order to ensure that all tags related to "name" are included in the same category or type.

### Street types
For auditing the types of street I produced a program similar to the one written in the case study lesson, which provides a dictionary with the different street types and all street names associated, as well as a dictionary with the total number of streets for each type. The code is available in the file **audit_street_types.py**.

In [6]:
import audit_street_types
import re

In [7]:
# Produce a regular expression to catch the first word on the street name
street_type_re = re.compile(r'\S+\b', re.IGNORECASE)

# Obtain all street types and the number of ocurrences for each type
street_types, street_types_count = audit_street_types.get_street_types(OSM_FILE, street_type_re)

In [8]:
print(street_types_count)

defaultdict(<class 'int'>, {'Rúa': 933, 'Praza': 104, 'Avenida': 76, 'Rolda': 4, 'Parque': 2, 'Glorieta': 3, 'Andén': 1, 'CARRETERA': 1, 'Travesía': 44, 'Praciña': 2, 'Calexón': 4, 'Estrada': 16, 'Paseo': 10, 'Xardíns': 1, 'Lugar': 39, 'Polígono': 10, 'Monte': 2, 'Camiño': 25, 'As': 2, 'Plaza': 1, 'Roi': 1, 'Cantón': 2, 'Escaleira': 1, 'Costa': 3, '2ª': 1, 'Bella': 1, 'Escalinata': 3, 'Agra': 2, 'Pasadizo': 2, 'Prolongación': 1, 'Boquete': 1, 'Estreita': 1, 'Vía': 1, 'Vereda': 2, 'Ruela': 2, 'Xardín': 1, 'Grupo': 4, 'Urbanización': 2, 'Eirís': 1, 'Campus': 3, 'Peirao': 2, 'Ponte': 1, 'Rosalía': 1, 'Anxo': 1, 'Av.Mallos': 1, 'Capitán': 1, 'Calle': 1, 'Avda': 1, 'Avd': 1, 'Carretera': 1, 'Santa': 1, 'María': 1, 'Elviña': 1, 'Zapateira': 1, 'Nostián': 1, 'Rampla': 1, 'Uxes': 1, 'Dársena': 1})


Looking at the street types, I observed several items which would need to be corrected. For clarity, I produced a list of expected street types and filtered them out in order to investigate the suspicious-looking items.

In [9]:
expected_street_types = ['Rúa', 'Praza', 'Avenida', 'Rolda', 'Glorieta', 'Travesía', 'Praciña', 'Estrada', 'Paseo', 'Lugar',
                         'Camiño', 'Cantón', 'Escaleira', 'Costa', 'Escalinata', 'Pasadizo', 'Estreita', 'Vía',
                         'Vereda', 'Calexón', 'Ruela', 'Peirao', 'Rampla', 'Autoestrada', 'Autovía']

In [10]:
# Print a sample of the non-standard items
i = 0
for key, values in street_types.items():
    if (key not in expected_street_types) and (i < 4):
        i += 1
        print(key, ':', values)

Parque : {'Parque de Santa Margarida', 'Parque de San Diego'}
Andén : {'Andén de Riazor'}
CARRETERA : {'CARRETERA NACIONAL VI KM. 589'}
Xardíns : {'Xardíns de Méndez Núñez'}


I identified the following issues which can be corrected programmatically:
* Items containing abbreviated street types: *Avd*, *Av.* or *Avda*.
* Items written in Spanish, whereas the official naming should be given in Galician, such as *Carretera* or *Calle*.
* Items containing the name of the street but not the street type &ndash; e.g. *Rosalía de Castro* or *Capitán Juan Varela*.

However, there are other issues whose correction is more difficult to automate:
* There are items which do not represent names of streets, but rather names of districts or neighborhoods, such as *Elviña*, *Uxes* or *Zapateira*.
* Some items refer to gardens or parks &ndash; e.g. *Xardín de San Carlos* &ndash; where it is difficult to know whether there is actually a street within this location with this name.

Since the correction of these issues would require the analysis of each case individually, I did not correct them in the processing of the data.

Additionally, I observed that some names contain the street type not in the first word, but in a later part of the street name &ndash; e.g. *2ª Travesía da Gaiteira*. Since there is no fixed rule for street naming, other than the abovementioned standard practice, there is no need to correct for this.

### Names of cities
Another issue which I came across while auditing the data is that the names of the cities provided in the OSM file are not entirely consistent:

In [11]:
# Print all the different cities found in nodes and ways
print(node_tag_types["addr:city"].union(way_tag_types["addr:city"]))

{'O Temple', 'O Temple, Cambre', 'Coruña', 'Arteixo', 'Santa Cristina', 'La Coruña', 'A Coruña', 'Culleredo', 'Santa Cruz', 'Oleiros', 'Cambre'}


There are three different cases of the name Coruña: A Coruña, Coruña and La Coruña. I will set all the cases to "A Coruña" when processing the data, as this is the official name of the city. Also, there are two cases for O Temple. Since this does not correspond to a city as such, but to a (sort of) neighborhood, I will correct it to "O Temple, Cambre", which contains the name of the city it belongs to.

### Postcodes
Finally, I reviewed the postal codes included in the dataset and, while the majority of them are consistent with the expectation &ndash; five-digit codes starting by 15, e.g. 15001 &ndash; there is a mismatching value: 1507. Since it is not possible to confirm the actual postcode of this case (it could be 15007 or any number between 15070 and 15079), I decided to remove this value from the dataset when converting the data to CSV format.

### Summary of corrections
To summarize, I list here the corrections applied to the data when converting the .osm file to CSV format:

* Search for street names in "name" tags and change their type to "name:street" if no "addr:street" tag is given for the associated node &ndash; unless the node is a bus stop, since bus stops usually are named after their address.
* Correct abbreviated street types, e.g. *Avenida* for *Av.*
* Correct street names given in Spanish to Galician.
* Add the street type to street names which do not contain it. (Here I will use the most common street type: *Rúa*)
* Correct names of cities containing A Coruña and O Temple.
* Discard the postal code if it does not contain a 5-digit code.

## 3. Data conversion to SQL
The first step in the conversion of the .osm file to a SQL database is to read and transform the data into CSV tables. For this, I created a Python program based on the one developed during the case study and applying the specific corrections for this map file. The Python file is **osm2csv.py**.

For the program to run I need to provide the following information:
* Name of the CSV files for the different tables to be created: *nodes*, *node_tags*, *ways*, *way_nodes* and *way_tags*.
* Dictionary containing the street names to be corrected according to the guidelines given above.

In [12]:
# Define the required CSV filenames
NODES_PATH = 'nodes.csv'
NODE_TAGS_PATH = 'node_tags.csv'
WAYS_PATH = 'ways.csv'
WAY_NODES_PATH = 'way_nodes.csv'
WAY_TAGS_PATH = 'way_tags.csv'

In [13]:
# Define a dictionary for the correction of street types and names
street_mapping = {'CARRETERA NACIONAL VI KM. 589': 'Estrada Nacional VI Km. 589', 'Plaza do Santuario': 'Praza do Santuario',
                  'Calle Cuba': 'Rúa Cuba', 'Carretera de A Zapateira': 'Estrada da Zapateira',
                  'Roi Xordo': 'Rúa Roi Xordo', 'Rosalía de Castro': 'Rúa Rosalía de Castro',
                  'Anxo senra Fernandez': 'Rúa Anxo Senra Fernández','Capitán Juan Varela': 'Rúa Capitán Juan Varela',
                  'María Corredoira': 'Rúa María Corredoira', 'Av.Mallos': 'Avenida dos Mallos',
                  'Avda Alfonso Molina': 'Avenida Alfonso Molina', 'Avd. Ernesto Che Guevara': 'Avenida Ernesto Che Guevara'}

In [14]:
import osm2csv

In [15]:
# Call function to process map file, apply corrections to data and save it into CSV files
osm2csv.process_map(OSM_FILE, street_mapping, expected_street_types, NODES_PATH, NODE_TAGS_PATH, WAYS_PATH, WAY_NODES_PATH, 
                    WAY_TAGS_PATH, validate=True)

For the generation of the SQL database I wrote a Python program based on the schema provided in the Udacity course (see ref. <a href="#ref1">[1]</a>) and the example given in ref. <a href="#ref2">[2]</a>. The code is provided in the file **csv2sqldb.py**.

In [16]:
import csv2sqldb

In [17]:
# Define name of SQL file and call function to create the database, using the CSV filenames defined above
SQL_DB_PATH = 'LCG_map.db'

csv2sqldb.create_database(SQL_DB_PATH, NODES_PATH, NODE_TAGS_PATH, WAYS_PATH, WAY_NODES_PATH, WAY_TAGS_PATH)

## 4. Database queries
For this section I made use of the method proposed in reference <a href="#ref3">[3]</a> for printing out SQL queries with Pandas, as well as some of the SQL code used in the sample project provided in the Udacity course (ref. <a href="#ref4">[4]</a>).

In [18]:
# Print file sizes
import os
file_list = [OSM_FILE, SQL_DB_PATH, NODES_PATH, NODE_TAGS_PATH, WAYS_PATH, WAY_NODES_PATH, WAY_TAGS_PATH]
for file in file_list:
    print(file, ':', str(round(float(os.path.getsize(file)/1e6), 2)), 'MB')

LCG_map.osm : 79.27 MB
LCG_map.db : 44.17 MB
nodes.csv : 26.14 MB
node_tags.csv : 4.76 MB
ways.csv : 2.68 MB
way_nodes.csv : 10.76 MB
way_tags.csv : 4.51 MB


In [19]:
# Import pandas for printing out SQL queries
import pandas as pd

# Create connection to SQL file
conn = csv2sqldb.create_connection(SQL_DB_PATH)

In [20]:
command = """SELECT COUNT(*) AS "Number of nodes" FROM nodes;"""
pd.read_sql_query(command, conn)

,Number of nodes
0,322004


In [21]:
command = """SELECT COUNT(*) AS "Number of ways" FROM ways;"""
pd.read_sql_query(command, conn)

,Number of ways
0,45788


In [22]:
command = """SELECT COUNT(DISTINCT(tables.uid)) AS "Number of unique users"
             FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) tables;"""
pd.read_sql_query(command, conn)

,Number of unique users
0,424


#### Number and type of shops

In [23]:
command = """SELECT tags.value, COUNT(*) AS Count 
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags
             WHERE tags.key == 'shop' GROUP BY tags.value ORDER BY count DESC LIMIT 5;"""
pd.read_sql_query(command, conn)

,value,Count
0,clothes,124
1,supermarket,118
2,hairdresser,69
3,bakery,55
4,car_repair,31


The type of shop with the highest presence in the database are clothes' shops, so I would like to see which clothing stores are the most common.

In [24]:
command = """SELECT nodes_tags.value, COUNT(*) as Count
             FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='clothes') i ON nodes_tags.id=i.id
             WHERE nodes_tags.key='name' GROUP BY nodes_tags.value ORDER BY Count DESC LIMIT 5;"""
pd.read_sql_query(command, conn)

,value,Count
0,Pull & Bear,3
1,Miss-tic,3
2,Zara,2
3,Women Secret,2
4,Vazva,2


Pull & Bear and Miss-tic are the two most common apparel stores in the map. However, it seems that the total number of shops is not truly representative of the reality as, for example, there are &ndash; to my knowledge &ndash; at least 5 Zara stores in the city. This might be a sign that the information contained in the OpenStreetMap of the city is incomplete, as not all stores of a given brand are included in it.

(As a side note, the reason for choosing Zara as an example is that the first ever Zara shop was opened in A Coruña in the 1970's and its parent company is based in the neighbor city of Arteixo, also present in the map data.)

## 5. Ideas for improvement
Seeing that the data relative to stores is not fully complete, my first proposal would be to complete this section of the map. Including the information of every single retailer in an entire city might be challenging and extremely time consuming, nevertheless, a lot of data should be easily accessible online, at least that related to medium or large shops.

In [25]:
command = """SELECT tags.value, COUNT(*) AS Count
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags 
              JOIN (SELECT DISTINCT(id) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) WHERE key='street' AND type='name') i
              ON tags.id=i.id WHERE tags.key='amenity' GROUP BY tags.value ORDER BY Count DESC;"""
pd.read_sql_query(command, conn)

,value,Count
0,taxi,12
1,parking,12
2,bicycle_rental,4


One of the issues I discovered during the data auditing process is that some nodes in the map contain street names and addresses tagged as "name". For this reason, I inspected the database to find the type of amenities which fall in this category. I observed that these points correspond to transport points, which could indicate that these transport nodes are named by their address &ndash; as I observed during the data auditing with the bus stops. However, further investigations showed me that there are other transport points which do contain their addresses tagged as "addr:street", creating an inconsistency in the data. Therefore, I think it should be feasible to correct this issue by setting all street names and addresses to tags such as "addr:street" and "addr:housenumber".

Finally, also during the data auditing, I observed that some places are tagged as "addr:street" even though they do not correspond to streets, such as parks and university campuses. I did not correct this issue, since it could not be done programmatically. Nevertheless, I believe these points should be assessed in order to correct any invalid data. As an example, I listed all the tags related to nodes where the "addr:street" is *Xardín de San Carlos*, which corresponds to a small garden next to my home. 

In [26]:
command = """SELECT *
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags 
              JOIN (SELECT DISTINCT(id) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) WHERE value='Xardín de San Carlos' AND type='addr') i
              ON tags.id=i.id LIMIT 10;"""
pd.read_sql_query(command, conn)

,id,key,value,type,id
0,3406247431,city,A Coruña,addr,3406247431
1,3406247431,housenumber,1,addr,3406247431
2,3406247431,postcode,15001,addr,3406247431
3,3406247431,source,Catastro 18-3-2015,regular,3406247431
4,3406247431,street,Xardín de San Carlos,addr,3406247431
5,75260454,alt_name,Arquivo Histórico de Galicia,name,75260454
6,75260454,city,A Coruña,addr,75260454
7,75260454,es,Archivo del Reino de Galicia,name,75260454
8,75260454,gl,Arquivo Histórico de Galiza,alt_name,75260454
9,75260454,gl,Arquivo do Reino de Galicia,name,75260454


I found out that there are two nodes with this tag: one of them corresponds to the garden itself (id = 3406247431), and the other one to the official archive of the region (id = 75260454). In the first case, I would simply change the tag containing the name of the park from "addr:street" to "name". For the second one, I would change the name of the street from *Xardín de San Carlos* to *Rúa San Carlos*, which is the actual street name where the building is located. 
This process would need to be performed for all the nodes and ways containing names of places in the place of street names. As mentioned before, this can only be done by analyzing each case individually, which would be highly time consuming.

## 6. Conclusions
To summarize, during this study I have examined and cleaned the OpenStreetMap data of my hometown, A Coruña. In doing this, I have observed that the map is incomplete in some parts and that it presents some issues, mainly related to language and formatting questions, but also that there are some points which contain invalid data, such as street names which do not exist.

All in all, however, it has been a very constructive exercise and, particularly interesting, was to see the large amount of information that is available for a relatively small city such as this one.

## 7. References
A list of websites and repositories that I have used and referenced in the text is provided in the file **REFERENCES.md**.